In [1]:
import random
import matplotlib.pyplot as plt 
import numpy as np
import struct
import matplotlib.pyplot as plt

def softmax(x):
    x_exp = np.exp(x-np.max(x,axis = 0))
    total = np.sum(x_exp,axis=0) # 列
    return x_exp / total
    

# 线性模型
def linear_model(w,b,x):
    return softmax(np.dot(w.T,x)+b)
# 交叉熵
def cross_entropy(y, y_hat):
    return - np.sum(y*np.log(y_hat+np.exp(-7)))

# 梯度下降
def sgd(w, b,x, y, alpha=0.01): # aaa
    batch_size = x.shape[1]
    lm = linear_model(w,b,x)
    dw = np.dot((y - lm),x.T)
    db = np.sum(y - lm, axis = 1,keepdims = True) # aaa
    w_o = w + alpha*dw.T/batch_size
    b_o = b + alpha*db/batch_size
    return w_o,b_o

def get_minibatch(epoch,batch_size,x,y):
    rounds = 67349 / batch_size
    index1 = int((epoch % rounds) * batch_size)
    index2 = int((epoch % rounds + 1) * batch_size)
    return x[:,index1 :index2], y[:,index1:index2]


x = np.load("sst-train-x.npy",allow_pickle=True) 
y = np.load("sst-train-y.npy",allow_pickle=True)
x_t  = np.load("sst-test-x.npy",allow_pickle=True)
y_t  = np.load("sst-test-y.npy",allow_pickle=True)
x = x.T.astype("float")
x_t  = x_t.T.astype("float")
y = y.T
y_t = y_t.T

# 画图
tstl = []
trnl = []
tacc = []
# 初始化数据

epochs = 10000 # aaa

w_dat = np.random.randint(1,100,size=(700,2))
w = w_dat / w_dat.sum(axis=0)
b = np.zeros((2,1))
'''

b = np.load("sst-linear_b.npy",allow_pickle = True)
w = np.load("sst-linear_w.npy",allow_pickle = True)
'''
batch_size = 67349 # aaa 67349
batch_size_t = 872

# 训练模型
for epoch in range(epochs):
    # 获得用于训练的minibatch
    minibatch_x,minibatch_y = get_minibatch(epoch,batch_size,x,y)
    # 初始化训练损失
    l = linear_model(w,b, minibatch_x)
    loss = cross_entropy(minibatch_y, l)
    # 累加每个元素的损失
    train_loss = loss / batch_size
    trnl.append(train_loss)
        
    # 梯度下降
    w,b = sgd(w,b,minibatch_x, minibatch_y)

    # 测试
    # 注意：测试集不需要梯度，也不需要梯度清零、反向传播、梯度下降
    l_t = linear_model(w,b, x_t)
    loss_t = cross_entropy(y_t, l_t)
    test_loss = loss_t / batch_size_t
    tstl.append(test_loss)
        
    # 累加计算正确的个数的平均值
    acc = (np.argmax(l_t,axis=0) == np.argmax(y_t,axis=0)).sum()
    tacc.append(acc / batch_size_t)

    # 每训练三次就输出一次性能
    if epoch % 100 == 0: # aaa
        train_y = linear_model(w,b, x)
        train_acc = (np.argmax(train_y,axis=0) == np.argmax(y,axis=0)).sum() / 67349
       # 输出
        print(f"\nTurn: {epoch // 1 + 1} -->\n"
            f"Train Loss: {train_loss / batch_size}\n"
            f"Test Loss: {test_loss / batch_size_t}\n"
            f"Train Accuracy: {train_acc}\n"
            f"Test Accuracy: {acc / batch_size_t}\n")
        np.save("sst-linear_w",w)
        np.save("sst-linear_b",b)

plt.figure(1)
plt.plot(tstl,color='b')#画在图1上
plt.plot(trnl,color='r')
plt.savefig('linear_loss_r.png')

plt.figure(2)
plt.plot()#画在图2上，且不在一个窗口              
plt.plot(tacc,color='b')
plt.savefig('linear_acc_r.png')
print("Over")



Turn: 1 -->
Train Loss: 1.0265072461520529e-05
Test Loss: 0.0007928679477987477
Train Accuracy: 0.5576474780620351
Test Accuracy: 0.5091743119266054

